In [ ]:
import xobjects as xo
import xtrack as xt
import xfields as xf
import xpart as xp

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import json
import time
from datetime import datetime
from scipy import constants as cst
import os

import sys
sys.path.append('/Users/pkicsiny/phd/cern/xsuite')
import input_files.config as config
import src.log as log
import src.init as init
import src.utils as utils
import src.plot_utils as plot_utils
import src.profiler as profiler

%load_ext wurlitzer

In [ ]:
xp.enable_pyheadtail_interface()
from PyHEADTAIL.feedback.transverse_damper import TransverseDamper
from PyHEADTAIL.impedances.wakes import WakeTable, WakeField
from PyHEADTAIL.machines.synchrotron import Synchrotron
from PyHEADTAIL.particles.slicing import UniformBinSlicer

In [ ]:
save_plots = True
save_date = datetime.today().strftime("%Y_%m_%d")

In [ ]:
which = "skekb_her"
beam_params = init.load_beam_params(which)

q0                  = beam_params[           "q_b1"]  # [e]
n_ip                = beam_params[           "n_ip"]  # [1]
bunch_intensity     = beam_params["bunch_intensity"]  # [1]
energy              = beam_params[         "energy"]  # [GeV]
p0c                 = beam_params[            "p0c"]  # [eV]
mass0               = beam_params[          "mass0"]  # [eV]
phi                 = beam_params[            "phi"]  # [rad] half xing
k2_factor           = beam_params[      "k2_factor"]  # [1]
qx                  = beam_params[             "Qx"]  # [1] half arc
qy                  = beam_params[             "Qy"]  # [1]
qs                  = beam_params[             "Qs"]  # [1]
physemit_x          = beam_params[     "physemit_x"]  # [m]
physemit_y          = beam_params[     "physemit_y"]  # [m]
beta_x              = beam_params[         "beta_x"]  # [m]
beta_y              = beam_params[         "beta_y"]  # [m]
sigma_x             = beam_params[        "sigma_x"]  # [m]
sigma_px            = beam_params[       "sigma_px"]  # [1]
sigma_y             = beam_params[        "sigma_y"]  # [m]
sigma_py            = beam_params[       "sigma_py"]  # [1]
sigma_z             = beam_params[        "sigma_z"]  # [m] sr
physemit_s          = beam_params[     "physemit_s"]  # [m]
sigma_delta         = beam_params[    "sigma_delta"]  # [1]
beta_s              = beam_params[         "beta_s"]  # [m]
gamma               = beam_params[          "gamma"]  # [1]
u_sr                = beam_params[           "U_SR"]  # [GeV/turn]

# emittance damping rates
damping_rate_s = 2*beam_params["damping_rate_s"]  # [turn] 2*u_sr/energy/n_ip
damping_rate_x = 2*beam_params["damping_rate_x"]  # [turn]
damping_rate_y = 2*beam_params["damping_rate_y"]  # [turn]

#u_bs                = beam_params[           "U_BS"]  # [GeV]


In [ ]:
context = xo.ContextCpu(omp_num_threads=8)

n_macroparticles_b1 = int(1e3)

n_slices = 100
n_turns = 10000

# WS
- first no beambeam
- linear lattice so no radiation model; effective synrad
- load longitudinal wakefield from file that Demin sent
- scan bunch intensity
- plot z distribution and see how it shifts due to increasing intensity + wakefield

## 2) arc with wakefield

In [ ]:
########################
# half arc with synrad #
########################

beta_x_sext_left  = 3
beta_y_sext_left  = 500
beta_x_sext_right = 3
beta_y_sext_right = 500

alpha_x_sext_left  = 0
alpha_y_sext_left  = 0
alpha_x_sext_right = 0
alpha_y_sext_right = 0


# between 2 sextupoles
el_arc_b1 = xt.LineSegmentMap(_context=context,
    qx =  qx, 
    qy =  qy,
    qs = -qs,
    betx = [ beta_x,  beta_x],
    bety = [ beta_y,  beta_y],
    alfx = [0, 0],
    alfy = [0, 0],
    bets = sigma_z/sigma_delta,
    damping_rate_x = damping_rate_x,
    damping_rate_y = damping_rate_y,
    damping_rate_s = damping_rate_s,
    equ_emit_x = physemit_x, 
    equ_emit_y = physemit_y, 
    equ_emit_s = physemit_s, # only here i need sigma_z delta SR
#    energy_increment = u_bs/n_ip*1e9, # U_BS for one IP
)

wakefile = "../../superkekb/wake/wakeLT_2021c_physics_woCSR_v2.2_LER.txt"
waketable = WakeTable(wakefile, ['time', 'dipole_x', 'dipole_y',
                                 'quadrupole_x', 'quadrupole_y', 'longitudinal'])

n_slices_wakes = 200
limit_z = 3 * sigma_z
slicer_for_wakefields = UniformBinSlicer(n_slices_wakes,
                                         z_cuts=(-limit_z, limit_z))

# <PyHEADTAIL.impedances.wake_kicks.ConstantWakeKickZ at 0x127ebdb40>
wake_field = WakeField(slicer_for_wakefields, waketable)

# Specity that the wakefield element needs to run on CPU and that lost
# particles need to be hidden for this element (required by PyHEADTAIL)
#wake_field.needs_cpu = True
#wake_field.needs_hidden_lost_particles = True

In [ ]:
bunchint_arr = np.array([1e8, 2e8, 5e8, 1e9, 2e9, 5e9, 1e10, 2e10, 5e10, 1e11])
len(bunchint_arr), bunch_intensity

In [ ]:
z_co_eq_arr     = []
delta_co_eq_arr = []
w = 500

for i, bunchint_i in enumerate(bunchint_arr):
    print(f"bunch_intensity: {bunchint_i:.2e}")
    
    #############
    # particles #
    #############
    
    particles_b1 = xp.Particles(
                _context = context, 
                q0        = q0,
                p0c       = p0c,
                mass0     = mass0,
                x         = sigma_x        *np.random.randn(n_macroparticles_b1),
                y         = sigma_y        *np.random.randn(n_macroparticles_b1),
                zeta      = 10*sigma_z        *np.random.randn(n_macroparticles_b1),
                px        = sigma_px       *np.random.randn(n_macroparticles_b1),
                py        = sigma_py       *np.random.randn(n_macroparticles_b1),
                delta     = 10*sigma_delta    *np.random.randn(n_macroparticles_b1),
                weight=bunchint_i/n_macroparticles_b1
                )
    
    particles_b1.name = "b1"
    particles_b1._init_random_number_generator()
    particles_b1.circumference = 1
    
    monitor = xt.ParticlesMonitor(particle_id_range=(0, n_macroparticles_b1),
                                  start_at_turn=0, stop_at_turn=n_turns)
    
    line = xt.Line(elements = [el_arc_b1, wake_field, monitor])
    line.build_tracker(_context=context)
    
    #########
    # track #
    #########
    
    measure_time = profiler.profiler()
    
    chunk_size = int(100)
    n_chunks = int(np.ceil(n_turns/chunk_size))
    
    for i_chunk in range(n_chunks):
    
        measure_time.start()
    
        line.track(particles_b1, num_turns=chunk_size)
        time_elapsed = measure_time.stop()
        
        alive_1     = np.sum(particles_b1.state[particles_b1.state==1])
        alive_1_idx = np.where(particles_b1.state==1)[0]
        y_std_1_aux = np.std(particles_b1.y[alive_1_idx], axis=0)
        less_than_7_sigma_in_alive_1_idx = np.where(np.abs(particles_b1.y[alive_1_idx]) <= y_std_1_aux * 7)[0]
        x_std_1     = np.std(particles_b1.x    [alive_1_idx][less_than_7_sigma_in_alive_1_idx], axis=0)
        y_std_1     = np.std(particles_b1.y    [alive_1_idx][less_than_7_sigma_in_alive_1_idx], axis=0)
        z_std_1     = np.std(particles_b1.zeta [alive_1_idx][less_than_7_sigma_in_alive_1_idx], axis=0)
        
        print(f"Chunk [{i_chunk+1}/{n_chunks}]: b1 alive: {alive_1}, σ_x,1: {x_std_1:.4e}, σ_y,1: {y_std_1:.4e}, σ_z,1: {z_std_1:.4e}, time: {time_elapsed:.4f}")
        
    ##################
    # get statistics #
    ##################
    
    coords_dict = monitor.to_dict()["data"]

    x_arr     = np.reshape(coords_dict[    "x"], (n_macroparticles_b1, n_turns))
    y_arr     = np.reshape(coords_dict[    "y"], (n_macroparticles_b1, n_turns))
    z_arr     = np.reshape(coords_dict[ "zeta"], (n_macroparticles_b1, n_turns))
    px_arr    = np.reshape(coords_dict[   "px"], (n_macroparticles_b1, n_turns))
    py_arr    = np.reshape(coords_dict[   "py"], (n_macroparticles_b1, n_turns))
    delta_arr = np.reshape(coords_dict["delta"], (n_macroparticles_b1, n_turns))
    
    x_co_arr     = np.mean(x_arr    , axis=0)    
    y_co_arr     = np.mean(y_arr    , axis=0)    
    z_co_arr     = np.mean(z_arr    , axis=0)    
    px_co_arr    = np.mean(px_arr   , axis=0)   
    py_co_arr    = np.mean(py_arr   , axis=0)   
    delta_co_arr = np.mean(delta_arr, axis=0)
    
    x_std_arr     = np.std(x_arr    , axis=0)    
    y_std_arr     = np.std(y_arr    , axis=0)    
    z_std_arr     = np.std(z_arr    , axis=0)    
    px_std_arr    = np.std(px_arr   , axis=0)   
    py_std_arr    = np.std(py_arr   , axis=0)   
    delta_std_arr = np.std(delta_arr, axis=0)
    
    emits_dict = monitor.to_dict()["data"]
    
    emit_x_arr, emit_y_arr, emit_s_arr = utils.stat_emittance_from_monitor(emits_dict, n_macroparticles_b1, n_turns, 
                                alpha_x=0,
                                alpha_y=0,
                                beta_x=beta_x,
                                beta_y=beta_y)
        
    #########
    # plots #
    #########
    
    save_path = "../plots/n121_skekb_wakefield_bunchint_scan"
    save_name = f"n121_{save_date}_{which}_{str(i).zfill(2)}_{bunchint_i:.2e}_nb"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    data_tuple_co   = (x_co_arr, y_co_arr, z_co_arr, px_co_arr, py_co_arr, delta_co_arr)
    params_tuple_co = (sigma_x, sigma_y, sigma_z, sigma_px, sigma_py, sigma_delta)
    fig1, ax1 = plot_utils.standard_plot_co(data_tuple_co, params_tuple_co, n_turns, n_ip, w=w, save_name=save_name, save_path=save_path)
    
    data_tuple_std   = (x_std_arr, y_std_arr, z_std_arr, px_std_arr, py_std_arr, delta_std_arr, emit_x_arr, emit_y_arr, emit_s_arr)
    params_tuple_std = (sigma_x, sigma_y, 10*sigma_z, sigma_px, sigma_py, 10*sigma_delta, physemit_x, physemit_y, 100*physemit_s)
    anal_z_tuple_std = (sigma_z, sigma_delta, physemit_s, damping_rate_s)
    fig2, ax2 = plot_utils.standard_plot_std(data_tuple_std, params_tuple_std, n_turns, n_ip, w=w, anal_z_tuple_std=anal_z_tuple_std, save_name=save_name, save_path=save_path)
    
    # hist
    fig3, ax3 = plt.subplots(1,1, figsize=(12, 10))
    ax3.hist(particles_b1.z/sigma_z, bins=100, color="b", alpha=.4);
    ax3.hist(particles_b1.delta/sigma_delta, bins=100, color="g", alpha=.4);
    
    z_co_eq     = np.mean(    z_co_arr[-w:])/     sigma_z
    delta_co_eq = np.mean(delta_co_arr[-w:])/ sigma_delta
    
    ax3.axvline(    z_co_eq, c="b", label=f"$<z>=${    z_co_eq:.4e}")
    ax3.axvline(delta_co_eq, c="g", label=f"$<δ>=${delta_co_eq:.4e}")
    
    ax3.legend(fontsize=14)
    ax3.set_xlabel(r"z, δ [$\mathrm{\sigma_{z,\delta,eq}}$]")
    ax3.set_ylabel("Count [1]")
    
    fig3.suptitle(f"{which}, $\mathrm{{N_b}}={bunchint_i:.2e}$, weak beam")
    fig3.tight_layout()
    fig3.savefig(os.path.join(save_path, save_name+"_co_hist.png"), bbox_inches="tight")
        
        
    z_co_eq_arr.append(z_co_eq)
    delta_co_eq_arr.append(delta_co_eq)
    
z_co_eq_arr = np.array(z_co_eq_arr)
delta_co_eq_arr = np.array(delta_co_eq_arr)


# Plots

In [ ]:
plt.plot(bunchint_arr, 100*z_co_eq_arr, "bo", label="<z>")
plt.plot(bunchint_arr, 100*delta_co_eq_arr, "ro", label="<δ>")

plt.xlabel("Bunch intensity [e]")
plt.ylabel(r"<z,δ> [$\mathrm{\sigma_{z,\delta,eq}}$, %]")
plt.grid()
plt.xscale("log")
plt.legend(fontsize=14)
plt.title(f"{which}\n$\mathrm{{N_m}}$={n_macroparticles_b1:.2e}, $\mathrm{{N_t}}$={n_turns:.2e}")
plt.tight_layout()

plt.savefig(os.path.join(save_path, f"n121_{save_date}_{which}_longitudinal_offset.png"))